## Import Relevant Packages

In [6]:
import os
import geopandas as gpd
import cartopy.crs as ccrs
import folium

## Import Data

In [7]:
substations = gpd.read_file(os.path.abspath('Data/belfastSubstations.shp')) # Import Substations data from Data folder within Repository
EVcharging = gpd.read_file(os.path.abspath('Data/evChargingStations_belfast.shp')) # Import EV Charging data from same folder

## Reproject Data to Projected CRS

In [8]:
substations_wm = substations.to_crs(epsg=3857) # Convert to Web Mercator
EVcharging_wm = EVcharging.to_crs(epsg=3857) # Convert to Web Mercator

## Create Buffer

In [9]:
buffer = substations_wm.buffer(200) # Create a 200m buffer around the Belfast Substations
buffer_gdf = gpd.GeoDataFrame(geometry=buffer) # Convert the GeoSeries into a GeoDataFrame
buffer_gdf.set_crs(epsg=3857, inplace=True) # Sets GDF to EPSG 3857

buffer_gdf.head() # Show the first 5 records within the newly created GeoDataFrame

,geometry
0,"POLYGON ((-655140.927 7283955.987, -655141.89 ..."
1,"POLYGON ((-663618.734 7288509.934, -663619.697..."
2,"POLYGON ((-665331.483 7277942.59, -665332.446 ..."
3,"POLYGON ((-660890.514 7282194.005, -660891.477..."
4,"POLYGON ((-658522.626 7283962.461, -658523.589..."


## Classify Charging Points

In [10]:
within_buffer = [] # Create an empty list to store inside or outside values

for point in EVcharging_wm.geometry: # Loop through all points within the EV Charging Points data
    is_within = buffer_gdf.intersects(point).any() # Check if point intersects buffer
    within_buffer.append(is_within) # Append result within list

EVcharging_wm["within_buffer"] = within_buffer # Adds list as column to dataset

## Create an inside and outside GeoDataFrame

In [11]:
EV_inside = EVcharging_wm[EVcharging_wm["within_buffer"]] # Create a gdf for points within a 200m buffer of a substation 
EV_outside = EVcharging_wm[~EVcharging_wm["within_buffer"]] # Create a gdf for points outside the 200m buffers

## Create Folium Map

In [18]:
m = buffer_gdf.explore(color="lightsalmon", popup=False, legend=True) # Create a folium map with buffer GDF added

## Add Additional Data

In [20]:
substations_wm.explore( m=m, marker_type="marker", color="dodgerblue", popup=True, legend=True)

EV_inside.explore( m=m, icon=folium.Icon(color="green", icon="car"), popup=True, legend=True)
EV_outside.explore( m=m, icon=folium.Icon(color="red", icon="car"), popup=True, legend=True)

## Save link

In [ ]:
m.save("EVcharging_Belfast.html")